# Задание 1.1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [1]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [2]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [3]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [4]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

In [5]:
def foldl2(f, x0, lst):
    return foldr(lambda x, y: lambda t: y(f(t, x)), lambda x: x, lst)(x0)

In [6]:
foldl2(f, 1, [1, 2, 3])

0.16666666666666666

In [7]:
def foldr2(f, x0, lst):
    return foldl(lambda y, x: lambda t: y(f(x, t)), lambda x: x, lst)(x0)

In [8]:
foldr2(f, 1, [1, 2, 3])

1.5

# Задание 1.2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

Функция проходит строку a с помощью подстроки a длины b. Последовательно с конца проверяется, входит ли какой либо элемент b в данную подстроку и если да, то удаляет из подстроки и строки b данный символ. Если в результате получили пустые списки (все символы совпали), то выводим True. Если какой либо символ подстроки не совпадает с каким либо оставшимся символом строки b, то данный символ и левее него отсекаются и рассматривается оставшаяся часть строки a.
В худшем случае алгоритм имеет сложность nm, где n - длина строки a, m - длина строки b

In [9]:
def inside(l_a, l_b, drop, res=False):
    for i in l_b:
        if i == l_a[-1]:
            drop-=1
            l_b.remove(i)
            
            if drop == 0:
                return 0 
            
            else:
                drop = inside(l_a[:-1], l_b, drop)

    return drop

In [10]:
def check_inv(a, b):   
    a_char = [x for x in a]
    b_char = [x for x in b]
    drop = len(b_char)
    print(a)
    print(b)

    while len(a_char) >= len(b_char):   
        res = inside(a_char[:len(b_char)], b_char.copy(), drop)
    
        
        if res == 0:
            return True
    
        else:
            a_char = a_char[res:]
    
    return False

In [11]:
a = 'abcrotm'
b = 'tro'

In [12]:
check_inv(a, b)

abcrotm
tro


True

# Задание 1.3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [13]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value

    def __iter__(self):
        if not self.left and not self.right:
            yield self.value
            
        if self.left:
            yield from self.left
            
        if self.right:
            yield from self.right

    
    def __str__(self):
        return f'{self.value}({self.left}, {self.right})'.replace('None', '*')

    def __repr__(self):
        return '{}({}, {}, {})'.format(self.__class__.__name__, 
                                       repr(self.value),
                                       repr(self.left) if self.left else '*',
                                       repr(self.right) if self.right else '*').replace(', *', '')


tree = Tree(0, Tree(1, Tree(3), Tree(4)),
            Tree(2))

list(tree) == [3, 4, 2]

True

In [14]:
print(iter(tree))
print(str(tree))
print(repr(tree))

<generator object Tree.__iter__ at 0x7fdb19086cf0>
0(1(3(*, *), 4(*, *)), 2(*, *))
Tree(0, Tree(1, Tree(3), Tree(4)), Tree(2))


# Задание 1.4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [15]:
import re

In [16]:
#Элементарные операции
def compute(a, b, opp):
    a = float(a)
    b = float(b)
    if opp == '+':
        return str(a + b)
    
    if opp == '-':
        return str(a - b)
    
    if opp == '*':
        return str(a * b)
    
    if opp == '\\':
        return str(a / b)
    

In [17]:
#Вычисление значений в скобках
def high_priority(string):

    if string[0] == '(':
        string = string[1:-1]
    
    #Вычисление операций первого приоритета
    while re.search(r" [*\\] ", string):   
        #Операция
        opp = re.search(r"[*\\]", string)[0]
        
        #Операнды 
        a, b = re.split(r" [*\\] " ,re.search(r"[+-]*[.\d]* [*\\] [+-]*[.\d]*", string)[0])
        
        #Замена операции на вычисленное значение
        string = re.sub(r"[+-]*[.\d]* [*\\] [+-]*[.\d]*", compute(a, b, opp), string)

    #Вычисление операций второго приоритета
    while re.search(r"\d+ [+-]", string):
        #Операция
        opp = re.search(r"\d+ [+-]", string)[0][-1]
        
        #Операнды 
        a, b = re.split(r" [+-] ", re.search(r"[+-]*[.\d]* [+-] [+-]*[.\d]*", string)[0])
        
        #Замена операции на вычисленное значение
        string = re.sub(r"[+-]*[.\d]* [+-] [+-]*[.\d]*", compute(a, b, opp), string)           
        
    return string
    

In [18]:
def calc(expr):
    try:
        while re.search(r"\(", expr):
            #Нахождение приоритетных скобок
            brackets = re.search(r"\([^()]*\)", expr)[0]
            
            #Вычисление
            expr = re.sub(r"\([^()]*\)", high_priority(brackets), expr)          
            expr = re.sub(r"--", "", expr)

        expr = high_priority(expr)       
        return float(expr)
    
    except ZeroDivisionError:
        return "Деление на ноль!"
    
    except:
        return "Неверный ввод!"

calc('2 * (15 - 3 * 4) - 2') == 4

#calc('2 * (15 - 3 * 4) - 2')
# calc('10 * ((15 - 5 * 4) - 2)')
# calc('10 * (-(-15 - 5 * 4) - 2)')
# calc('10 \ -((35 - 5 * 4) - 2)')
# calc('10 \ -((35 - 5 * 4) - 15)')
#calc('1 \ -((35 - 5 * 4) - 10)') == 1 / -((35 - 5 * 4) - 10)
#calc('10 \ -((35 - 5 * 4) - -10)')

True